In [0]:
%sql
select cast(key as string), cast(value as string)
from raw_bronze_kakfa r
where topic="orders"
limit 5

In [0]:
%sql
select v.* from (
select from_json(cast(value as string), "order_id string, order_timestamp Timestamp, customer_id string, quantity bigint, total bigint, books ARRAY<STRUCT<book_id string, quantity bigint, subtotal bigint>>") v
from raw_bronze_kakfa
where topic="orders")

In [0]:
from pyspark.sql import functions as F

json_schema = "order_id string , order_timestamp Timestamp, customer_id string , quantity bigint, total bigint, books array<struct<book_id string,quantity bigint,subtotal bigint>>"

query =(spark.readStream
             .table('raw_bronze_kakfa')
             .filter("topic='orders'")
             .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
             .select("v.*")
        .writeStream
             .option("checkpointLocation","/Volumes/workspace/bookstore_eng_pro/checkpoints/orders_silver")
             .trigger(availableNow=True)
             .table("orders_silver")
             )
query.awaitTermination()

In [0]:
%sql
select * from orders_silver limit 5